In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
url = 'https://www.opinion.com.bo/'

In [4]:
opinion = requests.get(url)
if opinion.status_code == 200:
    s = BeautifulSoup(opinion.text,'lxml')
    secciones = s.find('div', attrs={'class':'main-menu-border'}).find_all('li')
    links_secciones = list(set([url + seccion.a.get('href')[1:] for seccion in secciones]))
    print(links_secciones)
else:
    print('Error: problemas con la pagina')


['https://www.opinion.com.bo/blog/section/pais', 'https://www.opinion.com.bo/blog/section/avisos-necrologicos', 'https://www.opinion.com.bo/blog/section/salud', 'https://www.opinion.com.bo/blog/section/vida-de-hoy2', 'https://www.opinion.com.bo/blog/section/guerra-en-ucrania', 'https://www.opinion.com.bo/blog/section/informe-especial', 'https://www.opinion.com.bo/blog/section/buenanoche', 'https://www.opinion.com.bo/blog/section/cochabamba', 'https://www.opinion.com.bo/tag/escenario-politico', 'https://www.opinion.com.bo/blog/section/policial', 'https://www.opinion.com.bo/album/', 'https://www.opinion.com.bo/blog/section/mundo', 'https://www.opinion.com.bo/blog/section/escena-del-crimen', 'https://www.opinion.com.bo/video/', 'https://www.opinion.com.bo/blog/section/coronavirus', 'https://www.opinion.com.bo/blog/section/especiales', 'https://www.opinion.com.bo/opinion/', 'https://www.opinion.com.bo/blog/section/tendencias', 'https://www.opinion.com.bo/blog/section/colibrito', 'https://w

In [5]:
def obtener_articulos(soup):
    """
    Funcion que recibe un objeto Beautifulsoup de una seccion de una pagina 
    y devuelve una lista de URLs de cada nota en la seccion.
    """
    list_notes_links = []
    # se obtiene el listado de articulos principales
    try:
        list_articles = soup.find('div', attrs={'class':'archive-contents'}).find_all('article')
        for article in list_articles:
            if article.a:
                if 'https://www.opinion.com.bo' in article.a.get('href'):
                    list_notes_links.append(article.a.get('href'))
                else:
                    list_notes_links.append('https://www.opinion.com.bo' + article.a.get('href'))      
    except:
        print('No se pudo obtener los link de los articulos en:', soup.find('meta', attrs={'property':'og:url'}).get('content'))
        
    return list_notes_links

In [64]:
def obtener_info(s_nota, url):
    # se crea un dict vacio para llenarlo de información
    info_dict = {}
    # Extraemos la sección
    seccion = s_nota.find('body').get('data-category')
    if seccion:
        info_dict['seccion'] = seccion
    else:
        info_dict['seccion'] = None

    # Extraemos el titulo
    titulo = s_nota.find('h2', attrs={'class':'title'})
    if titulo:
        try:
            info_dict['titulo'] = titulo.text
        except Exception as e:
            info_dict['titulo'] = None
            print('Error extrayendo titulo: ', e)
    else:
        info_dict['titulo'] = None
        
    # Extraemos la fecha
    cont_fecha = s_nota.find('script', attrs={'type':'application/ld+json'})
    if cont_fecha:
        try:
            if cont_fecha.text.find('datePublished'):
                pos_ini = cont_fecha.text.find('datePublished') + len('datePublished') + 4
                pos_fin = pos_ini + len('2022-07-30T18:12:31-04:00')
                info_dict['fecha'] = cont_fecha.text[pos_ini:pos_fin]
            else:
                raise AttributeError('No se pudo encontrar la fecha')
        except AttributeError as ae:
            info_dict['fecha'] = None
            print('Error extrayendo fecha:')
            print(ae)
    else:
        info_dict['fecha'] = None

    # Extraemos el resumen
    resumen = s_nota.find('div', attrs={'class':'summary'})
    if resumen:
        try:
            info_dict['resumen'] = resumen.text[:-3]
        except Exception as e:
            info_dict['resumen'] = None
            print('Error extrayendo resumen: ',e)
    else:
        info_dict['resumen'] = None

    # Extraemos el autor de la nota
    autor = s_nota.find('span', attrs={'class':'author-name'})
    if autor: 
            try:
                info_dict['autor'] = autor.a.text[1:-1]
            except:
                try:
                    info_dict['autor'] = autor.text[1:-1]
                except Exception as e:
                    info_dict['autor'] = None
                    print('Error 1er y 2do comando: autor --> dato no extraido')
                    print(e)
    else:
        info_dict['autor'] = None
        
    # Extraemos el contenido
    contenido = s_nota.find('div', attrs={'class':'body'})
    try:
        if len(contenido.find_all('p')) > 1:
            contenido = contenido.find_all('p')
            contenido_unido = ''
            for parrafo in contenido:
                contenido_unido = contenido_unido + parrafo.text + ' '
            info_dict['texto'] = contenido_unido.replace('\xa0', ' ')
            
        elif len(contenido.find_all('p')) == 1:
            contenido = contenido.text.replace('\n\n', ' ').replace('\n','')
            info_dict['texto'] = contenido
        else:
            print(f'Contenido no encontrado en: {url}')
            info_dict['texto'] = None
            
    except Exception as e:
        info_dict['texto'] = None
        print('Error extrayendo contenido ',e)
        
    return info_dict

In [8]:
def scrapear_nota(url):
    try:
        nota = requests.get(url)
    except Exception as e:
        print('Error scrapeando la url: ', url)
        print(e)
        return None
    if nota.status_code != 200:
        print(f'Error obteniendo la nota: {url}')
        print(f'Status Code: {nota.status_code}')
        return None
    s_nota = BeautifulSoup(nota.text, 'lxml')
    
    info_dict = obtener_info(s_nota, url)
    info_dict['url'] = url
    
    return info_dict

In [10]:
notas = []
for link in links_secciones:
    try:
        r = requests.get(link)
        if r.status_code == 200:
            soup = BeautifulSoup(r.text, 'lxml')
            notas.extend(obtener_articulos(soup))
        else:
            print('No se pudo obtener la sección', link)
    except Exception as e:
        print('No se pudo obtener la seccion', link)

print(f'Recoleccion de notas terminada: {len(notas)} notas obtenidas')

No se pudo obtener los link de los articulos en: https://www.opinion.com.bo/album
No se pudo obtener los link de los articulos en: https://www.opinion.com.bo/video
No se pudo obtener los link de los articulos en: https://www.opinion.com.bo/opinion
Recoleccion de notas terminada: 527 notas obtenidas


In [12]:
notas

['https://www.opinion.com.bo/articulo/pais/camacho-cumbre-van-paro-48-horas-cierre-fronteras-abrogan-decreto-censal/20220730181231875244.html',
 'https://www.opinion.com.bo/articulo/pais/civicos-instituciones-instalan-cumbre-jhonny-fernandez-fue-invitado-arenga-paro/20220730160019875230.html',
 'https://www.opinion.com.bo/articulo/pais/disenan-manual-controlar-agencias-empleo-combatir-trata/20220730150634875228.html',
 'https://www.opinion.com.bo/articulo/pais/gobierno-plantea-especializar-policia-fiscalia-trata-trafico-personas/20220730143426875224.html',
 'https://www.opinion.com.bo/articulo/pais/aval-camacho-ratifican-cumbre-hoy/20220730120308875212.html',
 'https://www.opinion.com.bo/articulo/pais/arias-dicen-que-quieren-hacer-boletas-aimara-quechua-guarani-perdida-dinero/20220730111558875210.html',
 'https://www.opinion.com.bo/articulo/pais/carta-abierta-parra-pecado-trabajar-transparencia-sobrevivir-sufrir/20220730105223875207.html',
 'https://www.opinion.com.bo/articulo/pais/cal

In [65]:
data = []
for i, nota in enumerate(notas):
    print(f'Scrapeando nota {i+1}/{len(notas)}')
    data.append(scrapear_nota(nota))

Scrapeando nota 1/527
Scrapeando nota 2/527
Scrapeando nota 3/527
Scrapeando nota 4/527
Scrapeando nota 5/527
Scrapeando nota 6/527
Scrapeando nota 7/527
Scrapeando nota 8/527
Scrapeando nota 9/527
Scrapeando nota 10/527
Scrapeando nota 11/527
Scrapeando nota 12/527
Scrapeando nota 13/527
Scrapeando nota 14/527
Scrapeando nota 15/527
Scrapeando nota 16/527
Scrapeando nota 17/527
Scrapeando nota 18/527
Scrapeando nota 19/527
Scrapeando nota 20/527
Scrapeando nota 21/527
Scrapeando nota 22/527
Scrapeando nota 23/527
Scrapeando nota 24/527
Scrapeando nota 25/527
Scrapeando nota 26/527
Scrapeando nota 27/527
Scrapeando nota 28/527
Scrapeando nota 29/527
Scrapeando nota 30/527
Scrapeando nota 31/527
Scrapeando nota 32/527
Scrapeando nota 33/527
Scrapeando nota 34/527
Scrapeando nota 35/527
Scrapeando nota 36/527
Scrapeando nota 37/527
Scrapeando nota 38/527
Scrapeando nota 39/527
Scrapeando nota 40/527
Scrapeando nota 41/527
Scrapeando nota 42/527
Scrapeando nota 43/527
Scrapeando nota 44/5

In [66]:
len(data)

527

In [67]:
df = pd.DataFrame(data)

In [68]:
df.head()

,seccion,titulo,fecha,resumen,autor,texto,url
0,pais,Camacho y cumbre van a paro de 48 horas con ci...,2022-07-30T18:12:31-04:00,"""También le quiero decir a Luis Arce: no juegu...",Redacción,Si el Gobierno no abroga el decreto supremo qu...,https://www.opinion.com.bo/articulo/pais/camac...
1,pais,"Cívicos e instituciones instalan ""cumbre""; Fer...",2022-07-30T16:00:19-04:00,El Alcalde de Santa Cruz declaró la mañana de ...,Erbol,El movimiento interinstitucional por adelantar...,https://www.opinion.com.bo/articulo/pais/civic...
2,pais,"Diseñan un ""manual"" para controlar las agencia...",2022-07-30T15:06:34-04:00,La ministra Verónica Navia explicó que será un...,ABI,"La ministra de Trabajo, Verónica Navia, inform...",https://www.opinion.com.bo/articulo/pais/disen...
3,pais,Gobierno proyecta especializar a los policías ...,2022-07-30T14:34:26-04:00,El viceministro de Justicia y Derechos Fundame...,Urgente.bo,El viceministro de Justicia y Derechos Fundame...,https://www.opinion.com.bo/articulo/pais/gobie...
4,pais,"Con el aval de Camacho, ratifican otra ""cumbre...",2022-07-30T12:03:08-04:00,Será la segunda cumbre departamental que soste...,Redacción,"Esta tarde, desde las 15:00, algunas organizac...",https://www.opinion.com.bo/articulo/pais/aval-...


In [69]:
df.to_csv('notas_opinion.csv')